In [1]:
import httpx
import unicodedata
import pandas as pd

In [2]:
# 1. Fetch the feed
URL = "https://api.digikala.com/v1/incredible-offers/"
r = httpx.get(URL, timeout=httpx.Timeout(30.0))
r.raise_for_status()                                   # <-- network / API errors bubble up
products = r.json()["data"]["incredible_products_list"]["products"]


In [3]:

def price_int(p: dict, key: str) -> int | None:
    # گرفتن مقدار قیمت از دیکشنری default_variant['price']
    if p.get("default_variant") and p["default_variant"].get("price"):
        return p["default_variant"]["price"].get(key)
    return None

def strip_invisibles(text: str) -> str:
    # حذف کاراکترهای نامرئی در صورت نیاز (مثال ساده)
    return text.strip()

rows = []

for p in products:
    org = price_int(p, "rrp_price")
    incr = price_int(p, "selling_price")
    discount = None
    if p.get("default_variant") and p["default_variant"].get("price"):
        discount = p["default_variant"]["price"].get("discount_percent")
    
    rows.append(
        {
            "title": strip_invisibles(p["title_fa"]),
            "product_url": f"https://www.digikala.com{p['url']['uri']}",
            "org_price_toman": org,
            "selling_price_toman": incr,
            "discount_percent": discount if discount is not None else 0,
        }
    )

# مرتب‌سازی براساس درصد تخفیف از زیاد به کم
rows_sorted = sorted(rows, key=lambda x: x["discount_percent"], reverse=True)

# تبدیل به DataFrame
df = pd.DataFrame(rows_sorted)

# ذخیره به فایل اکسل
out_file = "products_sorted_by_discount.xlsx"
df.to_excel(out_file, index=False, engine="openpyxl")

print(f"✅  Saved {len(df)} rows to {out_file}")

✅  Saved 20 rows to products_sorted_by_discount.xlsx
